In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0002_heat_1d"
config.target = "jmlr"
config.debug_mode = True

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams.update(config.tueplots_bundle())

In [ ]:
rng = np.random.default_rng(24)

## Problem Definition

In [ ]:
spatial_domain = linpde_gp.domains.asdomain([-1.0, 1.0])

ibvp = linpde_gp.problems.pde.HeatEquationDirichletProblem(
    t0=0.0,
    T=5.0,
    spatial_domain=spatial_domain,
    alpha=0.1,
    initial_values=linpde_gp.functions.TruncatedSineSeries(
        spatial_domain,
        coefficients=[1.0],
    ),
)

In [ ]:
plt_grid = ibvp.domain.uniform_grid((50, 50))

## Prior

In [ ]:
lengthscale_t = 2.5
lengthscale_x = 2.0
output_scale = 1.0

In [ ]:
u_prior = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Zero(input_shape=(2,)),
    cov=output_scale ** 2 * linpde_gp.randprocs.covfuncs.TensorProduct(
        linpde_gp.randprocs.covfuncs.Matern((), nu=1.5, lengthscales=lengthscale_t),
        linpde_gp.randprocs.covfuncs.Matern((), nu=2.5, lengthscales=lengthscale_x),
    ),
)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], u_prior.mean(plt_grid))

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], ibvp.solution(plt_grid))

### Dirichlet Problem

In [ ]:
N_ic = 5
N_bc = 50

#### Initial conditions

In [ ]:
X_ic = ibvp.initial_domain.uniform_grid(N_ic, inset=1e-6)
Y_ic = ibvp.initial_condition.values(X_ic[..., 1])

u_ic = u_prior.condition_on_observations(Y_ic, X=X_ic)

#### Boundary Conditions

In [ ]:
u_ic_bc = u_ic

for bc in ibvp.boundary_conditions:
    X_bc = bc.boundary.uniform_grid(N_bc)
    Y_bc = bc.values(X_bc)

    u_ic_bc = u_ic_bc.condition_on_observations(Y_bc, X=X_bc)

#### Prior with Initial and Boundary Conditions

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], u_ic_bc.mean(plt_grid))

### Cauchy Problem

In [ ]:
N_ic = 3
N_bc = 10

In [ ]:
left_boundary_op = linpde_gp.linfuncops.diffops.DirectionalDerivative(direction=[0., 1.])
right_boundary_op = linpde_gp.linfuncops.diffops.DirectionalDerivative(direction=[0., -1.])

#### Initial Conditions

In [ ]:
X_ic = np.stack(
    (
        np.zeros(N_ic),
        np.linspace(*domain[1], N_ic),
    ),
    axis=-1,
)
Y_ic = bvp.boundary_conditions[0].values(X_ic[..., 1])

u_ic = u_prior.condition_on_observations(Y_ic, X_ic)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid_t, plt_grid_x, u_ic.mean(plt_grid))

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid_t, plt_grid_x, right_boundary_op(u_ic).mean(plt_grid))

#### Left Boundary Condition

In [ ]:
X_bc_left = np.stack(
    (
        np.linspace(*domain[0], N_bc + 1)[1:],
        np.full(N_bc, domain[1][0]),
    ),
    axis=-1
)
Y_bc_left = np.zeros_like(X_bc_left[..., 0])

In [ ]:
u_ic_bc_left = u_ic.condition_on_observations(
    Y_bc_left,
    X=X_bc_left,
    L=left_boundary_op,
)

#### Right Boundary Condition

In [ ]:
X_bc_right = np.stack(
    (
        np.linspace(*domain[0], N_bc + 1)[1:],
        np.full(N_bc, domain[1][1]),
    ),
    axis=-1,
)
Y_bc_right = np.zeros_like(X_bc_right[..., 0])

In [ ]:
u_ic_bc = u_ic_bc_left.condition_on_observations(
    Y_bc_right,
    X=X_bc_right,
    L=right_boundary_op,
)

#### Plots

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid_t, plt_grid_x, u_ic_bc.mean(plt_grid))

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid_t, plt_grid_x, right_boundary_op(u_ic_bc).mean(plt_grid))

### Conditioning on the PDE

In [ ]:
N_pde = (100, 20)

In [ ]:
# Before
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], ibvp.pde.diffop(u_ic_bc).mean(plt_grid))

In [ ]:
X_pde = ibvp.domain.uniform_grid(N_pde)
Y_pde = ibvp.pde.rhs(X_pde)

In [ ]:
u_ic_bc_pde = u_ic_bc.condition_on_observations(
    Y_pde,
    X=X_pde,
    L=ibvp.pde.diffop,
)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], u_ic_bc_pde.mean(plt_grid))
ax.set_xlabel("Time (s)")
ax.set_ylabel("Location (cm)")
ax.set_zlabel("Temperature (°C)")

experiment_utils.savefig("heat_posterior")

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], ibvp.pde.diffop(u_ic_bc_pde).mean(plt_grid))

In [ ]:
np.average(np.abs(u_ic_bc_pde.mean(plt_grid) - ibvp.solution(plt_grid)))

### Generate Animation

In [ ]:
plt.rcParams["font.weight"] = "light"
plt.rcParams["axes.labelweight"] = "light"
plt.rcParams["axes.titleweight"] = "light"

In [ ]:
import functools
from matplotlib import animation

fig, ax = plt.subplots()

ax.set_xlabel("Location (cm)")
ax.set_ylabel("Temperature (°C)")
ax.set_title("t = 0.00s")

@functools.partial(
    animation.FuncAnimation,
    fig,
    frames=plt_grid.shape[0],
    interval=10,
    repeat_delay=4000,
    blit=False,
)
def anim(frame_idx):
    txs = plt_grid[frame_idx, :, :]
    
    ax.cla()
    
    mean = u_ic_bc_pde.mean(txs)
    std = u_ic_bc_pde.std(txs)
    
    ax.plot(txs[:, 1], mean)
    ax.fill_between(
        txs[:, 1],
        mean - 1.96 * std,
        mean + 1.96 * std,
        alpha=.3,
    )

    ax.plot(txs[:, 1], ibvp.solution(txs))

    ax.set_ylim(-0.01, 1.2)
    ax.set_xlabel("Location (cm)")
    ax.set_ylabel("Temperature (°C) ")
    ax.set_title(f"t = {plt_grid[frame_idx, 0, 0]:.2f} s")


In [ ]:
from IPython.display import HTML

HTML(anim.to_jshtml())

In [ ]:
anim_path = experiment_utils.config.experiment_results_path / "heat_anim"

if anim_path.is_dir():
    import shutil
    
    shutil.rmtree(anim_path)

anim_path.mkdir(exist_ok=True)

anim.save(anim_path / "{}.pdf", linpde_gp.utils.plotting.PDFWriter())

In [ ]:
anim.save(
    experiment_utils.config.experiment_results_path / "heat_anim.gif",
    animation.PillowWriter(fps=5)
)